In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [3]:
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['day'] = data['context_timestamp_tmp'].dt.day
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [4]:
data = pd.read_csv('train.csv',index_col=False)
data2 = pd.read_csv('test.csv',index_col=False) 
data3 = pd.read_csv('test1.csv',index_col=False)

In [5]:
data = pre_process(data)
data3 = pre_process(data3)
data2 = pre_process(data2)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [11]:
data[['user_id','day','is_trade']].groupby(['user_id','day']).sum().reset_index()

,user_id,day,is_trade
0,24779788309075,19,0
1,36134987234568,22,0
2,59341486148291,22,0
3,179317972644611,22,0
4,179317972644611,23,0
5,212049717332865,21,0
6,217191929432135,22,0
7,218627666998650,18,0
8,225768902336823,18,0
9,300164043453670,22,0


In [9]:
data[['user_id','day','is_trade']].groupby(['user_id','day']).count()>10

is_trade
user_id             day          
24779788309075      19      False
36134987234568      22      False
59341486148291      22      False
179317972644611     22      False
                    23      False
212049717332865     21      False
217191929432135     22      False
218627666998650     18      False
225768902336823     18      False
300164043453670     22      False
317896740941222     21      False
343687024536467     23      False
343846706165657     23      False
471191193416668     24      False
506604787274365     23      False
510416141439025     19      False
523117691035117     24      False
547560177346870     21      False
601357023584293     18      False
635591985978049     21      False
636099518731252     21      False
643902588708579     20      False
654859444444842     21      False
735230298329321     20      False
                    21      False
740725272974461     24      False
804651294451073     18      False
808429707297646     20      False
                    23      False
810869007662288     19      False
...                           ...
9222280343957495772 22      False
9222375243455137760 19      False
9222418414773457583 23      False
9222476565403393762 20      False
                    22      False
9222500199833617909 24      False
9222513822757222000 19      False
9222553786417581634 18      False
                    19      False
                    23      False
                    24      False
9222567600393382449 18      False
                    24      False
9222660336380554559 21      False
9222661010396903891 20      False
9222741368620686467 21      False
9222790095407479120 20      False
9222795826502953393 21      False
9222878895135397908 19      False
9223077942117037524 18      False
9223099985366849305 18      False
9223214422415101680 19      False
                    20      False
9223262087638217435 19      False
9223263001143284782 19      False
9223286667125380235 22      False
9223296966028830603 18      False
9223324018512912014 23      False
9223335008361681580 24      False
9223345267743287634 19      False

[229457 rows x 1 columns]

In [12]:
all_data = data.append(data2)
all_data = all_data.append(data3)

In [13]:
user_click = all_data[['user_id','day','instance_id']].groupby(['user_id','day']).count()

In [14]:
user_click = user_click.rename(columns=({'instance_id':'day_click'}))

In [15]:
user_click = user_click.reset_index()

In [16]:
user_click

,user_id,day,day_click
0,7209196426168,25,1
1,24779788309075,19,3
2,36134987234568,22,1
3,59341486148291,22,1
4,174676690398750,25,1
5,179317972644611,22,7
6,179317972644611,23,1
7,179451404933115,25,8
8,193133497894650,25,1
9,212049717332865,21,1


In [17]:
userclickdata = pd.merge(data,user_click,on=['user_id','day'],how='left')

In [18]:
userclickdata2 = pd.merge(data2,user_click,on=['user_id','day'],how='left')

In [19]:
userclickdata3 = pd.merge(data3,user_click,on=['user_id','day'],how='left')

In [20]:
userclickdata[['instance_id','day_click']].to_csv('train_day_click.csv',index=False) 

In [21]:
userclickdata2[['instance_id','day_click']].to_csv('test_day_click.csv',index=False) 

In [22]:
userclickdata3[['instance_id','day_click']].to_csv('test1_day_click.csv',index=False) 